In [1]:
# Import libraries
from functions import *

import json
import os
import subprocess
import time

# Script level timelocks (OP_CLTV and OP_CSV)

In this section we'll cover how to create a transaction level timelocks with `OP_CHECKLOCKTIMEVERIFY` (`OP_CTLV`) and `OP_CHECKSEQUENCEVERIFY` (`OP_CSV`).

Script level timelocks are an essential part of many layer 2 protocols such lightning HTLC transactions and cross chain atomic swaps. They build off `nLocktime` and `nSequence` by allowing the script verification to check that the spending transaction has those fields set to specific values. 

This section assumes knowledge of P2WPKH transactions and transaction-level timelocks.

### Reading
- https://medium.com/summa-technology/bitcoins-time-locks-27e0c362d7a1

### Scenario
We will use the same scenario as with the transaction-level timelock example:

> Kim is an old grandmother who wants to pass some of her wealth (0.1 btc) on to her grandson, Peter. However, Peter is still a child and Kim doesn't trust Peter to spend his bitcoin wisely. Kim would rather wait until Peter turns 18 before he's able to spend his bitcoin. Kim doesn't know how much longer she'll be around for, and she doesn't trust anyone else to handle her money. To give Peter bitcoin in a way which will only allow him to spend it once he's 18, Kim will do this by giving Peter a signed transaction for 0.1 btc with an absolute timelock set to expire on his 18th birthday.

> Note that 1 year corresponds to about 50,000 blocks. To save us having to mine tens of thousands of regtest blocks (which may take a few minutes), we'll instead suppose Peter's 18th birthday is happening much sooner, at block 500.

In the previous notebook we achieved this by using transaction-level timelocks. There were a some potential downsides of this solution however:
- Although Peter has a valid signed transaction from Kim, it is possible that she, or someone with access to her private key, may spend the input before Peter is able to broadcast his transaction. 
- Since the timelocks were on a transaction level, any additional outputs would be encumbered by the same timelock. 
- Peter has to safely store the signed transaction until it can be broadcast.

A better solution in this case would be to use script-level timelocks. The OP_CODES `OP_CHECKLOCKTIMEVERIFY`  
and `OP_CHECKSEQUENCEVERIFY` are included in the redeemScript and can enforce that the spending transaction uses a specific value for `nLocktime` or `nSequence` respectively. 

### Create a script using OP_CHECKLOCKTIMEVERIFY

We'll create the script that Kim and Peter will agree on to send the bitcoin to. Peter wants to ensure that his public key is included in the script, so that he can create the signature for it, and Kim wants to be sure that the script has a timelock on it. 

In [2]:
# Set a public key for the receiver (Peter)
receiver_privkey = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
receiver_pubkey = privkey_to_pubkey(receiver_privkey)

# let's suppose Peter's 18th birthday is estimated to happen at block 500
locktime_int = 500
locktime = locktime_int.to_bytes(4, byteorder="little", signed=False)
locktime_val = locktime_int.to_bytes(2, byteorder="little", signed=False)

sequence = bytes.fromhex("ffff fffe")
print("sequence: ", sequence.hex())

redeemScript = (
    varint_len(locktime_val)
    + locktime_val
    + bytes.fromhex("b1") # OP_CLTV
    + bytes.fromhex("75") # OP_DROP
    + pushbytes(receiver_pubkey)
    + bytes.fromhex("ac"))

print(redeemScript.hex())

sequence:  fffffffe
02f401b1752102466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f27ac


#### Convert the redeemScript to a P2WSH address

For more on this step, review the 'Addresses' notebook.

In [3]:
address_to_spend = script_to_p2wsh(redeemScript, "regtest")
print(address_to_spend)

bcrt1qd75w96efgck0afp7zxarmhmzp2zjmzq0khkdf0fpaasrzl09m6aszmc5kn


#### Create Peter's utxo with 0.1 btc

Now that we have the address, we'll fund it using the python code we used in the previous sections, `create_regtest_utxo.py`.

In [4]:
txid_to_spend, index_to_spend = create_regtest_utxo(address_to_spend, 0.1)
print(f"txid: {txid_to_spend}, {index_to_spend}")

txid: 918ade7e41118455ab8029baa8baa0657e659461949d6798fd00b6f0cb88c156, 1


## Spending a P2WSH time-locked UTXO

Now that we have some funds locked up in a P2WSH utxo with a timelock, we can create a transaction spending from it. Let's say Peter wants to send 0.099 btc to the address `mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE`.

From our previous examples we know this corresponds to a scriptPubkey of `76a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac`.

In [5]:
output1_spk = bytes.fromhex("76a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac")
output1_value_sat = int(float("0.099") * 100000000)

### Create an unsigned transaction

Using the values for `nLocktime` that we set above `nSequence`, we'll create the unsigned transaction and sign it in the same way we did when creating a regular P2WSH transaction. Note that the value for `nLocktime` must be a 4 byte integer, whereas the value preceeding `OP_CHECKLOCKTIMEVERIFY` must be a variable length integer.

In [6]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# SEQUENCE
# set above

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
output_count = bytes.fromhex("01")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
)

# LOCKTIME
# set above

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  020000000156c188cbf0b600fd98679d946194657e65a0baa8ba2980ab558411417ede8a910100000000fffffffe01e00f9700000000001976a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88acf4010000


### Signing the transaction

Segwit transaction signing scheme described in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki)


In [7]:
input_amount_sat = int(0.1 * 100_000_000)
value = input_amount_sat.to_bytes(8, byteorder="little", signed=False)

hashPrevOuts = hash256(txid + index)
hashSequence = hash256(sequence)
hashOutputs = hash256(outputs)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + txid
    + index
    + varint_len(redeemScript)
    + redeemScript
    + value
    + sequence
    + hashOutputs
    + locktime
    + sighash_type
)
print(tx_digest_preimage.hex())

020000008439501705c1d2101354c585378a37abe081074f0447e9f751614b52db7e5fdfbbdebfac1cd12080fb29f8919f7f8be7bdb21a89c8d09e8a59fc7bb1d2a7379656c188cbf0b600fd98679d946194657e65a0baa8ba2980ab558411417ede8a91010000002802f401b1752102466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f27ac8096980000000000fffffffe5d18c7a03d8b14317752ff0d8801c9fe97f8e11c3abfc3d0543fa2fedcceb020f401000001000000


Hash this transaction and produce an ecdsa signature on it. 

In [8]:
# Create sigHash to be signed
sighash = hash256(tx_digest_preimage)

# Sign the sigHash with the input private key
signing_key = ecdsa.SigningKey.from_string(receiver_privkey, curve=ecdsa.SECP256k1) 
signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature = signature + bytes.fromhex("01")

# Witness field
witness = (
    # indicate the number of stack items
    bytes.fromhex("02")
    + pushbytes(signature)
    + pushbytes(redeemScript)
)

# tx_in with our new sigScript containing the signature we just created
inputs_signed = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs_signed
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_tx.hex())

signed transaction:  0200000000010156c188cbf0b600fd98679d946194657e65a0baa8ba2980ab558411417ede8a910100000000fffffffe01e00f9700000000001976a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac0247304402205dc2ec61433f9b5e54c24999329cd9e1d70a5666f5db5b8fb9a2f1e232e8210202204d9d5ac941831c8f60299ffd85c29eac860f2c4537d866136a625c8076d7b110012802f401b1752102466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f27acf4010000


### Scenario
The `OP_CHECKLOCKTIMEVERIFY` OP_CODE in the script is essentially forcing the transaction spending from it (Peter's transaction) to have `nLocktime` (and `nSequence` not set to `0xffffffff`) set in the transaction spending from it. Therefore Peter's transaction will behave as we expect for a transaction that has a transaction-level timelock.

If he were to try broadcasting the transaction at the current height (102), it would result in the following `non-final` error.

In [9]:
height = subprocess.getoutput("bitcoin-cli -regtest getblockcount ")
print(height)

invalid_attempt = subprocess.getoutput(f"bitcoin-cli -regtest testmempoolaccept '[\"{signed_tx.hex()}\"]'")
print(invalid_attempt)

102
[
  {
    "txid": "b6ba6901af8da67f36056d32a3839a3a44785b47e5262d5a6cb4924e74bbc0f9",
    "wtxid": "4689dfae0e66da7e90ed69d7f632859daae604c77000d524f9e50e3fddcb06c3",
    "allowed": false,
    "reject-reason": "non-final"
  }
]


Now we'll simulate 398 blocks passing (to reach block 500), and see what happens if Peter tries to broadcast his transaction again.

In [10]:
subprocess.getoutput(f"bitcoin-cli -regtest -generate 398")
height = subprocess.getoutput("bitcoin-cli -regtest getblockcount ")
print("blockheight: ", height)

valid_spend = subprocess.getoutput(f"bitcoin-cli -regtest testmempoolaccept '[\"{signed_tx.hex()}\"]'")
print(valid_spend)

blockheight:  500
[
  {
    "txid": "b6ba6901af8da67f36056d32a3839a3a44785b47e5262d5a6cb4924e74bbc0f9",
    "wtxid": "4689dfae0e66da7e90ed69d7f632859daae604c77000d524f9e50e3fddcb06c3",
    "allowed": true,
    "vsize": 114,
    "fees": {
      "base": 0.00100000
    }
  }
]


This time the transaction is successfully broadcasted and Peter has successfully spent his script-level, time-locked output. 

To view the decoded transaction, uncomment the lines below.

In [11]:
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + signed_tx.hex())
print(decoded)

{
  "txid": "b6ba6901af8da67f36056d32a3839a3a44785b47e5262d5a6cb4924e74bbc0f9",
  "hash": "4689dfae0e66da7e90ed69d7f632859daae604c77000d524f9e50e3fddcb06c3",
  "version": 2,
  "size": 201,
  "vsize": 114,
  "weight": 456,
  "locktime": 500,
  "vin": [
    {
      "txid": "918ade7e41118455ab8029baa8baa0657e659461949d6798fd00b6f0cb88c156",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "304402205dc2ec61433f9b5e54c24999329cd9e1d70a5666f5db5b8fb9a2f1e232e8210202204d9d5ac941831c8f60299ffd85c29eac860f2c4537d866136a625c8076d7b11001",
        "02f401b1752102466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f27ac"
      ],
      "sequence": 4278190079
    }
  ],
  "vout": [
    {
      "value": 0.09900000,
      "n": 0,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 3bc28d6d92d9073fb5e3adf481795eaf446bceed OP_EQUALVERIFY OP_CHECKSIG",
        "desc": "addr(mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE)#xlnzfr97",
   

In [13]:
# stop bitcoin core
subprocess.getoutput("bitcoin-cli -regtest stop")

'Bitcoin Core stopping'

## Script-level relative timelocks (OP_CHECKSEQUENCEVERIFY)

Similar to how `OP_CHECKLOCKTIMEVERIFY` enforces the transaction spending that output to set an absolute locktime with `nLocktime`, `OP_CHECKSEQUENCEVERIFY` enforces the transaction spending that output to set a relative timelock with `nSequence`.

In the scenario we just covered, Kim could have achieved the same outcome but using a relative timelock instead. The only difference would be that the delay would be specified relative to when the input was confirmed, rather than the absolute block height. 

Note that `version` must be set to **`2`** for relative timelocks to be enabled.

### Exercise
Repeat the scenario above, but using `OP_CHECKSEQUENCEVERIFY` to enforce the delay.

Hint: the block chain starts off at block 102, so the relative timelock should be set to 398 blocks for the transaction to be broadcastable at block 500.

Hint 2: nSequence is a 32 bit unsigned integer, so you can use the same `.to_bytes` method used to convert `locktime_int` to `locktime` in the previous example.


#### Solution for running with nSequence
```
locktime_int = 0
locktime = locktime_int.to_bytes(4, byteorder="little", signed=False)

sequence_int = 398
sequence = sequence_int.to_bytes(4, byteorder="little", signed=False)
sequence_val = sequence_int.to_bytes(2, byteorder="little", signed=False)

redeemScript = (
    varint_len(sequence_val)
    + sequence_val
    + bytes.fromhex("b2") # OP_CSV
    + bytes.fromhex("75") # OP_DROP
    + pushbytes(receiver_pubkey)
    + bytes.fromhex("ac"))

```

## Quiz

- 1. Earlier we learned that setting `nSequence` to `0xffffffff` disables `nLocktime`, so what would happen if you tried setting `nSequence` to `0xffffffff` when spending from an input that uses `OP_CLTV`?
- 2. What information does Peter need to hold on to in order to spend from the outputs in these examples?

## Answers
- 1. The transaction would be invalid as `OP_CLTV` checks that `nSequence` in the spending transaction is not set to `0xffffffff`.
- 2. He needs his private key to sign the transaction, and the redeem script.
